In [1]:
import sys
sys.path.append('../scripts')
import proplot as pplt
import pandas as pd
import cartopy.crs as ccrs
from helpers import *
from collections import Counter
pd.set_option('display.max_rows', 500)

def print(*val):
    display(*val)

In [2]:
all_buoys = get_metadata().index.to_list()

lengths = pd.DataFrame([{"sensor_id": buoy, "length": len(buoy_data(buoy, "original"))} for buoy in all_buoys])
lengths = lengths.set_index("sensor_id")

In [3]:
print(lengths.describe())

,length
count,216.000000
mean,8917.712963
std,9758.036525
min,4.000000
25%,2919.250000
50%,5698.000000
75%,10717.500000
max,52998.000000


In [4]:
freqs = get_metadata()["Calculated Frequency"].to_list()
print(Counter(freqs))

Counter({'60min': 100,
         '30min': 79,
         '120min': 24,
         '180min': 7,
         '240min': 4,
         '720min': 2})

In [12]:
valid_buoys = [buoy for buoy in get_metadata().index if usable_buoy(buoy)]

interp_lens = pd.DataFrame([{"sensor_id": buoy, "length": len(buoy_data(buoy, "interp"))} for buoy in valid_buoys])
interp_lens = interp_lens.set_index("sensor_id")

grouped_stations = [i for i in valid_buoys[["DN Station ID", "Buoy Type"]].groupby("DN Station ID")]


def most_data(df):
    return interp_lens.loc[df.index].idxmax().loc["length"]
def best_buoy(station_df):
    tbuoys = station_df[station_df["Buoy Type"] == "Thermistor"]
    if len(tbuoys) > 0:
        return most_data(tbuoys)
    else:
        return most_data(station_df)

{station[0]: best_buoy(station[1]) for station in grouped_stations}

TypeError: list indices must be integers or slices, not list